In [1]:
#This will be our sentinel processing script
# Import earthengine API
import ee
# Authenticate and initialise 
ee.Initialize()

In [2]:
import geemap

In [3]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")
sample_image = S2.filterBounds(ee.Geometry.Point(-71.0, 19.0)).sort('CLOUDY_PIXEL_PERCENTAGE').first()
#sample_image.getInfo()

In [4]:
def ESAcloudMask(img):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
     
    parsed by Nick Clinton
    """

    qa = img.select('QA60')

    # bits 10 and 11 are clouds and cirrus
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # both flags set to zero indicates clear conditions.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    # clouds is not clear
    cloud = clear.Not().rename(['ESA_clouds'])

    # return the masked and scaled data.
    return img.addBands(cloud)

In [5]:
haiti_region = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
  .filter(ee.Filter.eq('country_na', 'Haiti'))

dr_region = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
  .filter(ee.Filter.eq('country_na', 'Dominican Republic'))

In [6]:
#train a classifier on the corine europe data set using Sentinel 2
training_image = S2.filterBounds(ee.Geometry.Point(11.4, 52.58)).sort('CLOUDY_PIXEL_PERCENTAGE').first()
roi = training_image.geometry()
#roi = ee.Geometry.Rectangle([10.4,52.0,12.5,53.5])
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

Corine_image = ee.ImageCollection("COPERNICUS/CORINE/V20/100m").first().clip(roi)


Corine_coll = ee.ImageCollection("COPERNICUS/CORINE/V20/100m")
Pal = Corine_coll.select('landcover').get('landcover_class_palette')
Pal_names = Corine_coll.get('landcover_class_names')



combined_image = ee.ImageCollection([training_image.select(bands),Corine_image])

randomPoints = ee.FeatureCollection.randomPoints(roi,1000);
#print(randomPoints.getInfo())
label = 'landcover'

Corine_samples = Corine_image.sampleRegions(randomPoints,geometries=True)
S2_samples = training_image.select(bands).sampleRegions(Corine_samples)
#print(Corine_samples.getInfo())
#print(S2_samples.getInfo())
#print(Corine_samples.getInfo())

#training = training_image.select(bands).sampleRegions(Corine_samples,properties =[label],scale=100)


trained = ee.Classifier.smileCart().train(S2_samples, label, bands)

classified = training_image.select(bands).classify(trained)

#classified.getInfo()

In [7]:
# Create a map centred at (lat, lon).
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [8]:
sld_intervals = '''<RasterSymbolizer>
<ColorMap type="intervals" extended="false" >
<ColorMapEntry color="#E6004D" quantity="111" label="Artificial surfaces; urban fabric; continuous urban fabric"/>
<ColorMapEntry color="#FF0000" quantity="112" label="Artificial surfaces; urban fabric; discontinuous urban fabric"/>
<ColorMapEntry color="#CC4DF2" quantity="121" label="Artificial surfaces; industrial, commercial, and transport units; industrial or commercial units"/>
<ColorMapEntry color="#CC0000" quantity="122" label="Artificial surfaces; industrial, commercial, and transport units; road and rail networks and associated land"/>
<ColorMapEntry color="#E6CCCC" quantity="123" label="Artificial surfaces; industrial, commercial, and transport units; port areas"/>
<ColorMapEntry color="#E6CCE6" quantity="124" label="Artificial surfaces; industrial, commercial, and transport units; airports"/>
<ColorMapEntry color="#A600CC" quantity="131" label="Artificial surfaces; mine, dump, and construction sites; mineral extraction sites"/>
<ColorMapEntry color="#A64DCC" quantity="132" label="Artificial surfaces; mine, dump, and construction sites; dump sites"/>
<ColorMapEntry color="#FF4DFF" quantity="133" label="Artificial surfaces; mine, dump, and construction sites; construction sites"/>
<ColorMapEntry color="#FFA6FF" quantity="141" label="Artificial surfaces; artificial, non-agricultural vegetated areas; green urban areas"/>
<ColorMapEntry color="#FFE6FF" quantity="142" label="Artificial surfaces; artificial, non-agricultural vegetated areas; sport and leisure facilities"/>
<ColorMapEntry color="#FFFFA8" quantity="211" label="Agricultural areas; arable land; non-irrigated arable land"/>
<ColorMapEntry color="#FFFF00" quantity="212" label="Agricultural areas; arable land; permanently irrigated land"/>
<ColorMapEntry color="#E6E600" quantity="213" label="Agricultural areas; arable land; rice fields"/>
<ColorMapEntry color="#E68000" quantity="221" label="Agricultural areas; permanent crops; vineyards"/>
<ColorMapEntry color="#F2A64D" quantity="222" label="Agricultural areas; permanent crops; fruit trees and berry plantations"/>
<ColorMapEntry color="#E6A600" quantity="223" label="Agricultural areas; permanent crops; olive groves"/>
<ColorMapEntry color="#E6E64D" quantity="231" label="Agricultural areas; pastures; pastures"/>
<ColorMapEntry color="#FFE6A6" quantity="241" label="Agricultural areas; heterogeneous agricultural areas; annual crops associated with permanent crops"/>
<ColorMapEntry color="#FFE64D" quantity="242" label="Agricultural areas; heterogeneous agricultural areas; complex cultivation patterns"/>
<ColorMapEntry color="#E6CC4D" quantity="243" label="Agricultural areas; heterogeneous agricultural areas; land principally occupied by agriculture, with significant areas of natural vegetation"/>
<ColorMapEntry color="#F2CCA6" quantity="244" label="Agricultural areas; heterogeneous agricultural areas; agro-forestry areas"/>
<ColorMapEntry color="#80FF00" quantity="311" label="Forest and semi natural areas; forests; broad-leaved forest"/>
<ColorMapEntry color="#00A600" quantity="312" label="Forest and semi natural areas; forests; coniferous forest"/>
<ColorMapEntry color="#4DFF00" quantity="313" label="Forest and semi natural areas; forests; mixed forest"/>
<ColorMapEntry color="#CCF24D" quantity="321" label="Forest and semi natural areas; scrub and/or herbaceous vegetation associations; natural grasslands"/>
<ColorMapEntry color="#A6FF80" quantity="322" label="Forest and semi natural areas; scrub and/or herbaceous vegetation associations; moors and heathland"/>
<ColorMapEntry color="#A6E64D" quantity="323" label="Forest and semi natural areas; scrub and/or herbaceous vegetation associations; sclerophyllous vegetation"/>
<ColorMapEntry color="#A6F200" quantity="324" label="Forest and semi natural areas; scrub and/or herbaceous vegetation associations; transitional woodland-shrub"/>
<ColorMapEntry color="#E6E6E6" quantity="331" label="Forest and semi natural areas; open spaces with little or no vegetation; beaches, dunes, sands"/>
<ColorMapEntry color="#CCCCCC" quantity="332" label="Forest and semi natural areas; open spaces with little or no vegetation; bare rocks"/>
<ColorMapEntry color="#CCFFCC" quantity="333" label="Forest and semi natural areas; open spaces with little or no vegetation; sparsely vegetated areas"/>
<ColorMapEntry color="#000000" quantity="334" label="Forest and semi natural areas; open spaces with little or no vegetation; burnt areas"/>
<ColorMapEntry color="#A6E6CC" quantity="335" label="Forest and semi natural areas; open spaces with little or no vegetation; glaciers and perpetual snow"/>
<ColorMapEntry color="#A6A6FF" quantity="411" label="Wetlands; inland wetlands; inland marshes"/>
<ColorMapEntry color="#4D4DFF" quantity="412" label="Wetlands; inland wetlands; peat bogs"/>
<ColorMapEntry color="#CCCCFF" quantity="421" label="Wetlands; maritime wetlands; salt marshes"/>
<ColorMapEntry color="#E6E6FF" quantity="422" label="Wetlands; maritime wetlands; salines"/>
<ColorMapEntry color="#A6A6E6" quantity="423" label="Wetlands; maritime wetlands; intertidal flats"/>
<ColorMapEntry color="#00CCF2" quantity="511" label="Water bodies; inland waters; water courses"/>
<ColorMapEntry color="#80F2E6" quantity="512" label="Water bodies; inland waters; water bodies"/>
<ColorMapEntry color="#00FFA6" quantity="521" label="Water bodies; marine waters; coastal lagoons"/>
<ColorMapEntry color="#A6FFE6" quantity="522" label="Water bodies; marine waters; estuaries"/>
<ColorMapEntry color="#E6F2FF" quantity="523" label="Water bodies; marine waters; sea and ocean"/>
</ColorMap>
</RasterSymbolizer>
'''

In [9]:
Map.addLayer(training_image, {'min':0,'max':2000,'bands': ['B4', 'B3', 'B2']}, 'image')
Map.addLayer(Corine_image.select('landcover'), {}, 'landcover')
Map.addLayer(randomPoints, {}, 'random points')
Map.addLayer(classified.sldStyle(sld_intervals),{},'classification')

In [10]:
Map.addLayer(haiti_region, {'color':'red'}, 'haiti_region')
Map.addLayer(dr_region, {'color':'blue'}, 'dr_region')

In [11]:
#Get composite image of S2 imagery over Hispianiola between jan-mar 2020 (filtered to cloud % <10)
HispComposite = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(ee.Geometry.Polygon(
        [[[-74.60668900677238, 20.088333497505946],
          [-74.60668900677238, 17.687361057873034],
          [-68.11376908489738, 17.687361057873034],
          [-68.11376908489738, 20.088333497505946]]])).filterDate('2020-01-10', '2020-03-20').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

#Get median image from the image collection (to reduce cloud impact)
HispCompositeMed = HispComposite.median()

#Run classification over whole island 
Hisp_classified = HispCompositeMed.select(bands).classify(trained)

#Define individual ROIs for Haiti and DR 
Haiti_roi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Haiti'))
DR_roi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Dominican Republic'))

#Clip classified image to each country (Haiti and DR)
Haiti_classified = Hisp_classified.clip(Haiti_roi)
DR_classified = Hisp_classified.clip(DR_roi)

In [12]:
Map.addLayer(sample_image,{'min':0,'max':2000,'bands':['B4','B3','B2']},'rbg')
Map.addLayer(HispCompositeMed, {'min':0,'max':2000,'bands':['B4','B3','B2']},'Hispaniola_med')
Map.addLayer(Hisp_classified.sldStyle(sld_intervals), {}, 'classified_hisp')
Map.addLayer(Haiti_classified.sldStyle(sld_intervals), {}, 'classified_Haiti')
Map.addLayer(DR_classified.sldStyle(sld_intervals), {}, 'classified_DR')

In [20]:
# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy = trained.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

Resubstitution error matrix:  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Training overall accuracy:  1


In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
array = trainAccuracy.getInfo()
df_cm = pd.DataFrame(np.array(array), index = list(range(513)),
                  columns = list(range(513)))
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

<AxesSubplot:>

In [21]:
# Sample the input with a different random seed to get validation data.
validation = training_image.select(bands).sample(**{
  'numPixels': 5000,
  'seed': 42
  # Filter the result to get rid of any {} pixels.
})

# Classify the validation data.
validated = validation.classify(trained)

# Get a confusion matrix representing expected accuracy.
testAccuracy = validated.errorMatrix('Land_Cover_Type_1', 'classification')
print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

Validation error matrix:  [[0]]
Validation overall accuracy:  0


In [29]:
dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2012');
landCover = dataset.select('landcover');

names = landCover.get('landcover_class_names').getInfo()
values = landCover.get('landcover_class_values').getInfo()
palette = landCover.get('landcover_class_palette').getInfo()

In [ ]:
for n,v,p in zip(names, values, palette):
    print(f'<ColorMapEntry color="#{p}" quantity="{v}" label="{n}"/>')